# Análisis Interactivo de Obras Públicas - GCBA

Esta notebook se conecta a tablas de Google Sheets, carga datos de obras públicas, los cruza con información geoespacial de barrios populares de CABA y genera análisis y visualizaciones.

## 1. Carga de Librerías

In [ ]:
# --- 1. Carga de Librerías ---
# Estas son las herramientas que usaremos para el análisis.
# Previamente ejecutar install_packages.R para instalar las librerías necesarias.

library(googlesheets4) # Nueva librería para conectar con Google Sheets
library(dplyr)       # Para manipulación de datos
library(ggplot2)     # Para gráficos estáticos
library(sf)          # Para trabajar con datos geoespaciales
library(leaflet)     # Para mapas interactivos
library(lubridate)   # Para manejo de fechas
library(cowplot)     # Para combinar gráficos
library(htmlwidgets) # Para guardar los widgets HTML como el mapa

## 2. Autenticación con Google

In [ ]:
# --- 2. Autenticación con Google ---
# La primera vez que ejecutes esto, se abrirá un navegador para iniciar sesión
# y autorizar a R a leer tus hojas de cálculo.
gs4_auth()

## 3. Carga de datos desde Google Sheets y Preprocesamiento

In [ ]:
# --- 3. Carga de datos desde Google Sheets y Preprocesamiento ---
# Pega aquí la URL completa de tu hoja de cálculo de Google.
url_google_sheet <- "https://docs.google.com/spreadsheets/d/xxxxxxxxxxxxxxxxxxxxxxxxxx/edit"

# Leemos cada hoja en un dataframe separado
df_obras <- read_sheet(url_google_sheet, sheet = "obras")
df_gastos <- read_sheet(url_google_sheet, sheet = "gastos_obra")

# Preprocesamiento para separar la columna LatLong
# Reemplaza "nombre_columna_unificada" con el nombre real de tu columna en Google Sheets
df_obras <- df_obras %>%
  separate(
    col = nombre_columna_unificada,   # La columna que tiene "-34.65, -58.39"
    into = c("latitud", "longitud"),  # Los nombres de las dos nuevas columnas que crearemos
    sep = ",",                        # El caracter que usaremos para separar (la coma)
    convert = TRUE                    # Convierte automáticamente el texto a números
  )

# Preprocesamiento de datos numéricos
df_obras <- df_obras %>%
  mutate(
    presupuesto_adjudicado = as.numeric(presupuesto_adjudicado),
    gastos_acumulados = as.numeric(gastos_acumulados)
  )

# Mostramos las primeras filas para verificar
print("Datos de Obras cargados:")
head(df_obras)

print("Datos de Gastos cargados:")
head(df_gastos)

## 4. Carga de datos geoespaciales (desde archivo local)

In [ ]:
# --- 4. Carga de datos geoespaciales (desde Archivo Local) ---
# Leemos el archivo GeoJSON de los barrios populares directamente desde el repositorio.

# Nombre del archivo que subiste a tu repositorio
archivo_geojson <- "env_baplvi_mi.json"

barrios_populares <- read_sf(archivo_geojson)

# Verificamos la estructura de los datos cargados
print("Datos de barrios populares cargados exitosamente:")
print(barrios_populares)

## 5. Análisis descriptivo

In [ ]:
# --- 5. Análisis descriptivo ---
# Realizamos cálculos básicos para obtener una visión general del estado de las obras.

presupuesto_total <- sum(df_obras$presupuesto_adjudicado, na.rm = TRUE)
gasto_total <- sum(df_obras$gastos_acumulados, na.rm = TRUE)
porcentaje_ejecutado <- (gasto_total / presupuesto_total) * 100

# Imprimimos un resumen formateado
cat("--- Resumen Financiero General ---\n")
cat(paste("Presupuesto Total Adjudicado: $", format(round(presupuesto_total, 2), big.mark = ".", decimal.mark = ","), "\n"))
cat(paste("Gasto Total Acumulado: $", format(round(gasto_total, 2), big.mark = ".", decimal.mark = ","), "\n"))
cat(paste("Porcentaje de Ejecución Presupuestaria:", round(porcentaje_ejecutado, 2), "%\n\n"))

# Contamos cuántas obras hay en cada estado
obras_por_estado <- df_obras %>%
  group_by(estado_obra) %>%
  summarise(Cantidad = n()) %>%
  ungroup()

print("Cantidad de obras por estado:")
print(obras_por_estado)

## 6. Visualización de datos (Gráficos estáticos)

In [ ]:
# --- 6. Visualización de datos (gráficos estáticos) ---
# Creamos gráficos para entender mejor la distribución del presupuesto y los tipos de obra.

# Gráfico 1: Presupuesto por Contratista
grafico_contratistas <- ggplot(df_obras, aes(x = reorder(contratista, -presupuesto_adjudicado), y = presupuesto_adjudicado)) +
  geom_bar(stat = "identity", fill = "#1A5276") +
  labs(title = "Presupuesto Adjudicado por Contratista", x = "Contratista", y = "Presupuesto (en $)") +
  scale_y_continuous(labels = scales::dollar_format(prefix = "$", big.mark = ".", decimal.mark = ",")) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 8), plot.title = element_text(hjust = 0.5, face = "bold"))

# Gráfico 2: Cantidad de Obras por Tipo
grafico_tipo_obra <- ggplot(df_obras, aes(x = reorder(tipo_obra, -table(tipo_obra)[tipo_obra]), fill = tipo_obra)) +
  geom_bar() +
  labs(title = "Cantidad de Obras por Tipo", x = "Tipo de Obra", y = "Cantidad") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 8), plot.title = element_text(hjust = 0.5, face = "bold"), legend.position = "none")

# Combinamos ambos gráficos en una sola imagen
graficos_combinados <- plot_grid(grafico_contratistas, grafico_tipo_obra, ncol = 2)

# Guardamos la imagen en el repositorio (útil para el README.md)
ggsave("analisis_obras_graficos.png", plot = graficos_combinados, width = 12, height = 6, dpi = 300)

# Mostramos los gráficos combinados aquí mismo en la notebook
graficos_combinados

## 7. Visualización geoespacial (Mapa interactivo)

In [ ]:
# --- 7. Visualización Geoespacial (Mapa Interactivo) ---
# Creamos un mapa interactivo con Leaflet para ubicar las obras y superponer los polígonos de los barrios populares.

obras_con_coords <- df_obras %>%
  filter(!is.na(latitud) & !is.na(longitud) & latitud != 0 & longitud != 0)

if (nrow(obras_con_coords) > 0) {
  # Preparamos el texto para los popups del mapa, incluyendo la foto
  obras_con_coords <- obras_con_coords %>%
    mutate(popup_text = paste0("<b>Obra:</b> ", nombre_obra, "<br>",
                               "<b>Contratista:</b> ", contratista, "<br>",
                               "<b>Estado:</b> ", estado_obra, "<br>",
                               "<b>Presupuesto:</b> $", format(round(presupuesto_adjudicado, 2), big.mark = ".", decimal.mark = ","), "<br>",
                               "<a href='", foto_principal_url, "' target='_blank'>Ver Foto</a>")) # Enlace a la foto

  # Creamos el mapa Leaflet
  mapa_obras <- leaflet(obras_con_coords) %>%
    addProviderTiles(providers$CartoDB.Positron, group = "Mapa Simple") %>%
    addProviderTiles(providers$Esri.WorldImagery, group = "Satélite") %>%
    
    # Añadimos los polígonos de barrios populares
    addPolygons(data = barrios_populares,
                fillColor = "#F39C12",
                fillOpacity = 0.5,
                color = "#F39C12",
                weight = 1.5,
                popup = ~paste0("<b>Barrio Popular:</b> ", nam), # Usamos la columna 'nam' del GeoJSON
                group = "Barrios Populares") %>%

    # Añadimos las obras como marcadores agrupados
    addCircleMarkers(~longitud, ~latitud,
                     radius = 6,
                     color = "#E74C3C",
                     fillOpacity = 0.8,
                     stroke = TRUE,
                     popup = ~popup_text,
                     clusterOptions = markerClusterOptions(),
                     group = "Obras") %>%
    
    # Añadimos el control de capas
    addLayersControl(
      baseGroups = c("Mapa Simple", "Satélite"),
      overlayGroups = c("Barrios Populares", "Obras"),
      options = layersControlOptions(collapsed = false)
    )
  
  # Guardamos el mapa interactivo como un archivo HTML
  saveWidget(mapa_obras, file = "mapa_obras_interactivo.html", selfcontained = TRUE)
  
  # Mostramos el mapa aquí mismo en la notebook
  mapa_obras

} else {
  print("No hay obras con coordenadas válidas para mostrar en el mapa.")
}